In [28]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from xgboost import XGBRegressor
import lightgbm as lgb
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, make_scorer
import numpy as np
from datetime import timedelta
import joblib # Importar joblib

In [29]:
df_final = pd.read_csv('data\df_tratado.csv')

In [30]:
print("\n=== Início da Seleção e Avaliação Inicial de Modelos ===")

markdown_cols_existing = ['MarkDown1', 'MarkDown2', 'MarkDown3', 'MarkDown4', 'MarkDown5']
existing_markdown_cols = [col for col in markdown_cols_existing if col in df_final.columns]


#Preparando os dados para o modelo
y = df_final['Weekly_Sales']
sample_weights = df_final['Sample_Weight']

features_to_use = [
    'Store', 'Dept', 'Size', 'Temperature', 'Fuel_Price', 'CPI', 'Unemployment',
    'IsHoliday_Flag',
    'Year', 'Month', 'Week', 'Day', 'DayOfWeek', 'DayOfYear',
    'SuperBowl', 'LaborDay', 'Thanksgiving', 'Christmas',
    'TotalMarkDown', 'HasAnyMarkDown'
]

for col in existing_markdown_cols:
    if f'Has_{col}' in df_final.columns and f'Has_{col}' not in features_to_use:
        features_to_use.append(f'Has_{col}')
    if col in df_final.columns and col not in features_to_use:
        features_to_use.append(col)

X = df_final[features_to_use]

categorical_features = ['Store', 'Dept']

preprocessor = ColumnTransformer(
    transformers=[
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ],
    remainder='passthrough'
)


=== Início da Seleção e Avaliação Inicial de Modelos ===


In [31]:
#Dividindo os dados para Treino e Teste
df_final_sorted = df_final.sort_values(by='Date').reset_index(drop=True)
X_sorted = df_final_sorted[features_to_use]
y_sorted = df_final_sorted['Weekly_Sales']
sample_weights_sorted = df_final_sorted['Sample_Weight']

train_size = int(len(df_final_sorted) * 0.8)
X_train_raw, X_test_raw = X_sorted.iloc[:train_size], X_sorted.iloc[train_size:]
y_train, y_test = y_sorted.iloc[:train_size], y_sorted.iloc[train_size:]
sample_weights_train, sample_weights_test = sample_weights_sorted.iloc[:train_size], sample_weights_sorted.iloc[train_size:]

print(f"\nDados divididos temporalmente:")
print(f"Tamanho do conjunto de treino: {X_train_raw.shape[0]} amostras")
print(f"Tamanho do conjunto de teste: {X_test_raw.shape[0]} amostras")
print(f"Período de treino: {df_final_sorted['Date'].iloc[0]} a {df_final_sorted['Date'].iloc[train_size-1]}")
print(f"Período de teste: {df_final_sorted['Date'].iloc[train_size]} a {df_final_sorted['Date'].iloc[-1]}")

X_train = preprocessor.fit_transform(X_train_raw)
X_test = preprocessor.transform(X_test_raw)

if isinstance(X_train, np.ndarray):
    pass
else:
    X_train = X_train.toarray()
    X_test = X_test.toarray()

print(f"\nShape de X_train após pré-processamento: {X_train.shape}")
print(f"Shape de X_test após pré-processamento: {X_test.shape}")


Dados divididos temporalmente:
Tamanho do conjunto de treino: 208238 amostras
Tamanho do conjunto de teste: 52060 amostras
Período de treino: 2011-02-04 a 2012-06-29
Período de teste: 2012-06-29 a 2012-10-26

Shape de X_train após pré-processamento: (208238, 148)
Shape de X_test após pré-processamento: (52060, 148)


In [32]:
#Função para calcular e avaliar as métricas
def evaluate_model(y_true, y_pred, model_name="Modelo"):
    mae = mean_absolute_error(y_true, y_pred)
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mape = np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print(f"\n=== Métricas de Avaliação para {model_name} ===")
    print(f"MAE: {mae:.2f}")
    print(f"RMSE: {rmse:.2f}")
    print(f"MAPE: {mape:.2f}%")
    return mae, rmse, mape

In [33]:
# === Adicionando a Regressão Linear Simples como Baseline ===
print("\nIniciando treinamento e avaliação do modelo de Regressão Linear (Baseline)...")
linear_model = LinearRegression()
linear_model.fit(X_train, y_train, sample_weight=sample_weights_train)
y_pred_linear = linear_model.predict(X_test)
mae_linear, rmse_linear, mape_linear = evaluate_model(y_test, y_pred_linear, "Regressão Linear")



Iniciando treinamento e avaliação do modelo de Regressão Linear (Baseline)...

=== Métricas de Avaliação para Regressão Linear ===
MAE: 8045.77
RMSE: 12257.81
MAPE: 8565.40%


In [34]:
#Treinamento do XGBoost Regressor
print("\nIniciando treinamento e avaliação do XGBoost Regressor...")
xgb_model = XGBRegressor(random_state=42, n_jobs=-1)
xgb_model.fit(X_train, y_train, sample_weight=sample_weights_train)
y_pred_xgb = xgb_model.predict(X_test)
mae_xgb, rmse_xgb, mape_xgb = evaluate_model(y_test, y_pred_xgb, "XGBoost Regressor")



Iniciando treinamento e avaliação do XGBoost Regressor...

=== Métricas de Avaliação para XGBoost Regressor ===
MAE: 3414.25
RMSE: 5525.24
MAPE: 4011.90%


In [35]:
#Treinamento do LightGBM Regressor
print("\nIniciando treinamento e avaliação do LightGBM Regressor...")
lgbm_model = lgb.LGBMRegressor(random_state=42, n_jobs=-1)
lgbm_model.fit(X_train, y_train, sample_weight=sample_weights_train)
y_pred_lgbm = lgbm_model.predict(X_test)
mae_lgbm, rmse_lgbm, mape_lgbm = evaluate_model(y_test, y_pred_lgbm, "LightGBM Regressor")


Iniciando treinamento e avaliação do LightGBM Regressor...
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.008845 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2948
[LightGBM] [Info] Number of data points in the train set: 208238, number of used features: 147
[LightGBM] [Info] Start training from score 16854.559579

=== Métricas de Avaliação para LightGBM Regressor ===
MAE: 4383.45
RMSE: 6726.22
MAPE: 5076.13%


d:\Projetos\ChallengeDS\.venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but LGBMRegressor was fitted with feature names
  warnings.warn(


In [36]:
#Comparação dos modelos
print("\n=== Comparação Final de Modelos ===")
print(f"Regressão Linear MAPE: {mape_linear:.2f}%")
print(f"XGBoost MAPE: {mape_xgb:.2f}%")
print(f"LightGBM MAPE: {mape_lgbm:.2f}%")

#Melhor modelo com base no MAPE
best_mape = min(mape_linear, mape_xgb, mape_lgbm)
if best_mape == mape_linear:
    best_model_name = "Regressão Linear"
elif best_mape == mape_xgb:
    best_model_name = "XGBoost"
else:
    best_model_name = "LightGBM"

print(f"\nO modelo '{best_model_name}' apresentou o melhor MAPE inicial ({best_mape:.2f}%).")

print("\n=== Seleção e Avaliação Inicial de Modelos Concluída ===")


=== Comparação Final de Modelos ===
Regressão Linear MAPE: 8565.40%
XGBoost MAPE: 4011.90%
LightGBM MAPE: 5076.13%

O modelo 'XGBoost' apresentou o melhor MAPE inicial (4011.90%).

=== Seleção e Avaliação Inicial de Modelos Concluída ===


In [37]:
print("\n=== Início da Otimização de Hiperparâmetros do Modelo (XGBoost) ===")

#Criando um scorer customizado para MAPE
def mape_scorer(y_true, y_pred):
    return -np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape_neg_scorer = make_scorer(mape_scorer, greater_is_better=True)


#busca de hiperparâmetros para XGBoost
param_dist = {
    'n_estimators': [100, 200, 300, 400], # Número de árvores
    'learning_rate': [0.01, 0.05, 0.1, 0.2], # Taxa de aprendizado
    'max_depth': [3, 5, 7, 9], # Profundidade máxima da árvore
    'subsample': [0.6, 0.8, 1.0], # Fração de amostras por árvore
    'colsample_bytree': [0.6, 0.8, 1.0], # Fração de features por árvore
    'gamma': [0, 0.1, 0.2], # Mínima redução de perda para uma divisão
    'reg_alpha': [0, 0.1, 0.5], # L1 regularization
    'reg_lambda': [0.5, 1, 1.5] # L2 regularization
}


=== Início da Otimização de Hiperparâmetros do Modelo (XGBoost) ===


In [38]:
#Inicializando o modelo base
xgb_base_model = XGBRegressor(random_state=42, n_jobs=-1)

random_search = RandomizedSearchCV(
    estimator=xgb_base_model,
    param_distributions=param_dist,
    n_iter=50, # Reduzido para 50 iterações para um teste inicial mais rápido. 200 travou localmente.
    cv=3,  
    scoring=mape_neg_scorer,
    verbose=2,
    random_state=42,
    n_jobs=-1   #Usando todos os núcleos da CPU para paralelizar
)

print(f"\nIniciando busca RandomizedSearchCV com {random_search.n_iter} iterações e {random_search.cv}-fold CV...")



Iniciando busca RandomizedSearchCV com 50 iterações e 3-fold CV...


In [39]:
#Otimização
random_search.fit(X_train, y_train, sample_weight=sample_weights_train)

print("\n=== Otimização Concluída ===")

# Obter os melhores parâmetros e o melhor modelo
best_xgb_params = random_search.best_params_
best_xgb_model = random_search.best_estimator_

print("\nMelhores Hiperparâmetros Encontrados para XGBoost:")
print(best_xgb_params)

d:\Projetos\ChallengeDS\.venv\Lib\site-packages\sklearn\model_selection\_search.py:881: UserWarning: The scoring make_scorer(mape_scorer, response_method='predict') does not support sample_weight, which may lead to statistically incorrect results when fitting RandomizedSearchCV(cv=3,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric=None, feature_types=None,
                                          feature_weights=None, gamma=None,
                                          grow_policy=None,
                     

Fitting 3 folds for each of 50 candidates, totalling 150 fits

=== Otimização Concluída ===

Melhores Hiperparâmetros Encontrados para XGBoost:
{'subsample': 0.6, 'reg_lambda': 1.5, 'reg_alpha': 0, 'n_estimators': 400, 'max_depth': 9, 'learning_rate': 0.2, 'gamma': 0.1, 'colsample_bytree': 1.0}


In [40]:
#Avaliando o Melhor Modelo no conjunto de teste
print("\nAvaliação do Melhor Modelo XGBoost Encontrado (no conjunto de teste):")
y_pred_best_xgb = best_xgb_model.predict(X_test)
mae_best_xgb, rmse_best_xgb, mape_best_xgb = evaluate_model(y_test, y_pred_best_xgb, "XGBoost Otimizado")

print(f"\nMAPE Inicial do XGBoost (não otimizado): {15057.52:.2f}%") # Relembrando o MAPE do card anterior
print(f"MAPE Otimizado do XGBoost: {mape_best_xgb:.2f}%")

print("\n=== Otimização de Hiperparâmetros do Modelo Concluída ===")


Avaliação do Melhor Modelo XGBoost Encontrado (no conjunto de teste):

=== Métricas de Avaliação para XGBoost Otimizado ===
MAE: 1998.32
RMSE: 3483.95
MAPE: 2276.73%

MAPE Inicial do XGBoost (não otimizado): 15057.52%
MAPE Otimizado do XGBoost: 2276.73%

=== Otimização de Hiperparâmetros do Modelo Concluída ===


In [41]:
print("\n=== Início da Geração das Previsões para o Próximo Ano ===")

#Combinações únicas de Loja/Departamento
unique_store_dept = df_final[['Store', 'Dept']].drop_duplicates().sort_values(by=['Store', 'Dept']).reset_index(drop=True)
print(f"Total de combinações únicas de Loja/Departamento: {len(unique_store_dept)}")



=== Início da Geração das Previsões para o Próximo Ano ===
Total de combinações únicas de Loja/Departamento: 2983


In [42]:
# O último ano nos dados é 2012 (dado que o dataset original vai até 26/10/2012).
# Vamos prever para o período imediatamente após a última data até completar 52 semanas.
df_final['Date'] = pd.to_datetime(df_final['Date'])

last_date_in_data = df_final['Date'].max()
# ==============================

prediction_start_date = last_date_in_data + timedelta(weeks=1) 
prediction_end_date = prediction_start_date + timedelta(weeks=52)

# O dataset original parece ter datas de sexta-feira.
future_dates = pd.date_range(start=prediction_start_date, end=prediction_end_date, freq='W-FRI')

print(f"Datas de previsão: de {future_dates.min().strftime('%Y-%m-%d')} a {future_dates.max().strftime('%Y-%m-%d')}")


Datas de previsão: de 2012-11-02 a 2013-11-01


In [43]:
#Juntando num dataframe com todas as combinações de loja/depto e datas futuras
future_df = pd.DataFrame()
for store_dept_idx, row in unique_store_dept.iterrows():
    temp_df = pd.DataFrame({'Date': future_dates,
                            'Store': row['Store'],
                            'Dept': row['Dept']})
    future_df = pd.concat([future_df, temp_df], ignore_index=True)

print(f"\nDataFrame futuro de base criado. Shape: {future_df.shape}")
print(future_df.head())


DataFrame futuro de base criado. Shape: (158099, 3)
        Date  Store  Dept
0 2012-11-02      1     1
1 2012-11-09      1     1
2 2012-11-16      1     1
3 2012-11-23      1     1
4 2012-11-30      1     1


In [44]:
#Features Temporais
future_df['Year'] = future_df['Date'].dt.year
future_df['Month'] = future_df['Date'].dt.month
future_df['Week'] = future_df['Date'].dt.isocalendar().week.astype(int)
future_df['Day'] = future_df['Date'].dt.day
future_df['DayOfWeek'] = future_df['Date'].dt.dayofweek
future_df['DayOfYear'] = future_df['Date'].dt.dayofyear

In [45]:
#Features de Feriado (IsHoliday_Flag, SuperBowl, LaborDay, Thanksgiving, Christmas)

future_df['IsHoliday'] = False # Default para não-feriado
# As datas exatas de feriados variam por ano, mas as semanas são consistentes.
# Vamos assumir que a coluna IsHoliday do arquivo features já reflete os feriados nos anos passados.
# Para o futuro, podemos usar uma heurística baseada nas datas históricas de IsHoliday=True
# Uma forma robusta seria ter um calendário de feriados exato para 2013, mas para este exercício, replicaremos a lógica.

holiday_weeks_months = df_final[df_final['IsHoliday'] == True][['Month', 'Week']].drop_duplicates()

future_df['IsHoliday'] = future_df.apply(
    lambda row: True if any((row['Month'] == hw['Month']) and (row['Week'] == hw['Week']) for idx, hw in holiday_weeks_months.iterrows()) else False,
    axis=1
)
future_df['IsHoliday_Flag'] = future_df['IsHoliday'].astype(int)

#Flags de feriados específicos (SuperBowl, LaborDay, Thanksgiving, Christmas)
future_df['SuperBowl'] = ((future_df['Month'] == 2) & (future_df['Week'].isin([6, 7])) & (future_df['IsHoliday'] == True)).astype(int)
future_df['LaborDay'] = ((future_df['Month'] == 9) & (future_df['Week'].isin([36])) & (future_df['IsHoliday'] == True)).astype(int)
future_df['Thanksgiving'] = ((future_df['Month'] == 11) & (future_df['Week'].isin([47])) & (future_df['IsHoliday'] == True)).astype(int)
future_df['Christmas'] = ((future_df['Month'] == 12) & (future_df['Week'].isin([51, 52])) & (future_df['IsHoliday'] == True)).astype(int)

In [46]:
#Features Econômicas/Climáticas (Temperature, Fuel_Price, CPI, Unemployment)
#Usando a média histórica por semana do ano
#Calculando as médias do dataset de treino para evitar vazamento
historical_weekly_avg = df_final.groupby('Week')[['Temperature', 'Fuel_Price', 'CPI', 'Unemployment']].mean().reset_index()

future_df = pd.merge(future_df, historical_weekly_avg, on='Week', how='left')
print("\nFeatures econômicas/climáticas preenchidas com médias históricas por semana.")

#Features MarkDown (TotalMarkDown, HasAnyMarkDown, Has_MarkDownX)
#Assumindo 0 para todas as promoções no futuro para uma previsão baseline
for col in markdown_cols_existing:
    future_df[col] = 0.0
    future_df[f'Has_{col}'] = 0 
df_final['TotalMarkDown'] = df_final[existing_markdown_cols].sum(axis=1)
future_df['TotalMarkDown'] = 0.0
future_df['HasAnyMarkDown'] = 0


Features econômicas/climáticas preenchidas com médias históricas por semana.


In [47]:
#Features da Loja ('Size', 'Type')
df_stores = pd.read_csv('D:/Projetos/ChallengeDS/data/raw/stores data-set.csv')
future_df = pd.merge(future_df, df_stores[['Store', 'Type', 'Size']], on='Store', how='left')

if 'Type' in future_df.columns:
    future_df['Type'] = future_df['Type'].astype('category')


# Organizando as colunas do future_df para corresponder à ordem das features no treinamento
# X_sorted.columns contém as features na ordem esperada pelo preprocessor
# X_train_raw é um dataframe, então tem nomes de colunas que preprocessor aprendeu.
# Devemos garantir que future_df_processed tenha as mesmas colunas na mesma ordem
future_X_raw = future_df[features_to_use]

print("\nFeatures futuras geradas e preparadas. Exemplo:")
print(future_X_raw.head())
print(f"Shape de future_X_raw: {future_X_raw.shape}")
print(f"Colunas de future_X_raw: {future_X_raw.columns.tolist()}")


Features futuras geradas e preparadas. Exemplo:
   Store  Dept    Size  Temperature  Fuel_Price         CPI  Unemployment  \
0      1     1  151315    48.532614    3.522661  172.664150      7.767738   
1      1     1  151315    48.737653    3.491016  172.741792      7.771354   
2      1     1  151315    51.750548    3.489257  172.856435      7.766710   
3      1     1  151315    49.113817    3.435670  172.965630      7.767530   
4      1     1  151315    45.616886    3.395459  173.092511      7.771769   

   IsHoliday_Flag  Year  Month  ...  Has_MarkDown1  MarkDown1  Has_MarkDown2  \
0               0  2012     11  ...              0        0.0              0   
1               0  2012     11  ...              0        0.0              0   
2               0  2012     11  ...              0        0.0              0   
3               1  2012     11  ...              0        0.0              0   
4               0  2012     11  ...              0        0.0              0   

   Mark

In [48]:
#Pré-processamento
#Usando o preprocessor JÁ FITADO nos dados de treino.
X_future_processed = preprocessor.transform(future_X_raw)

if hasattr(X_future_processed, 'toarray'):
    X_future_processed = X_future_processed.toarray()

print(f"\nShape de X_future_processed após pré-processamento: {X_future_processed.shape}")



Shape de X_future_processed após pré-processamento: (158099, 148)


In [49]:
#Previsões
print("\nGerando previsões de vendas com o modelo XGBoost otimizado...")
future_predictions = best_xgb_model.predict(X_future_processed)

future_predictions[future_predictions < 0] = 0
print("\nPrevisões geradas e valores negativos ajustados para 0.")



Gerando previsões de vendas com o modelo XGBoost otimizado...

Previsões geradas e valores negativos ajustados para 0.


In [50]:
#Formatando a saída
df_future_predictions = future_df[['Store', 'Dept', 'Date']].copy()
df_future_predictions['Predicted_Weekly_Sales'] = future_predictions

print("\n=== Exemplo de Previsões Geradas ===")
print(df_future_predictions.head(10)) #as primeiras 10 previsões

print(f"\nTotal de previsões geradas: {len(df_future_predictions)}")
print(f"Período das previsões: {df_future_predictions['Date'].min()} a {df_future_predictions['Date'].max()}")

print("\n=== Geração das Previsões para o Próximo Ano Concluída ===")


=== Exemplo de Previsões Geradas ===
   Store  Dept       Date  Predicted_Weekly_Sales
0      1     1 2012-11-02            38250.226562
1      1     1 2012-11-09            34385.386719
2      1     1 2012-11-16            33070.886719
3      1     1 2012-11-23            35606.609375
4      1     1 2012-11-30            34301.164062
5      1     1 2012-12-07            33047.320312
6      1     1 2012-12-14            42374.613281
7      1     1 2012-12-21            40778.207031
8      1     1 2012-12-28            34227.746094
9      1     1 2013-01-04            19969.082031

Total de previsões geradas: 158099
Período das previsões: 2012-11-02 00:00:00 a 2013-11-01 00:00:00

=== Geração das Previsões para o Próximo Ano Concluída ===


In [51]:
print("\n=== Início da Serialização e Salvamento do Modelo e Pré-processador ===")

models_dir = 'models'
os.makedirs(models_dir, exist_ok=True) 

model_path = os.path.join(models_dir, 'best_xgb_model.joblib')
preprocessor_path = os.path.join(models_dir, 'preprocessor.joblib')

print(f"Salvando o modelo XGBoost otimizado em: {model_path}")
joblib.dump(best_xgb_model, model_path)
print("Modelo salvo com sucesso.")

print(f"Salvando o pré-processador em: {preprocessor_path}")
joblib.dump(preprocessor, preprocessor_path)
print("Pré-processador salvo com sucesso.")


=== Início da Serialização e Salvamento do Modelo e Pré-processador ===
Salvando o modelo XGBoost otimizado em: models\best_xgb_model.joblib
Modelo salvo com sucesso.
Salvando o pré-processador em: models\preprocessor.joblib
Pré-processador salvo com sucesso.


In [52]:
print("\nVerificando se os arquivos podem ser carregados de volta...")
try:
    loaded_model = joblib.load(model_path)
    loaded_preprocessor = joblib.load(preprocessor_path)
    print("Modelo e pré-processador carregados com sucesso para verificação.")

    print(f"Tipo do modelo carregado: {type(loaded_model)}")
    print(f"Tipo do pré-processador carregado: {type(loaded_preprocessor)}")

except Exception as e:
    print(f"Erro ao carregar os arquivos: {e}")
    print("Isso pode indicar um problema com o salvamento.")

print("\n=== Serialização e Salvamento do Modelo e Pré-processador Concluída ===")


Verificando se os arquivos podem ser carregados de volta...
Modelo e pré-processador carregados com sucesso para verificação.
Tipo do modelo carregado: <class 'xgboost.sklearn.XGBRegressor'>
Tipo do pré-processador carregado: <class 'sklearn.compose._column_transformer.ColumnTransformer'>

=== Serialização e Salvamento do Modelo e Pré-processador Concluída ===
